# Lab 10
### DSCI 510 / USC / Hermjakob section / Spring 2025
---
Hello and welcome to DSCI 510's Lab10.

Guidelines for the Lab:

- You must complete the assignments individually. If you have trouble completing the assignment, please let the teaching assistant (TA) know, during the lab or their office hours. He will help and guide you, but he will not write code for you and no one else should :) !!!  

- You have to fill in the code in this notebook and upload it back to Brightspace for submission. Please remember to rename your file as "Lab10_[YOUR FIRSTNAME]_[YOUR LASTNAME].ipynb" (e.g. Lab10_George_Washington.ipynb).

- You may look up resources online like python docs and stackoverflow. You may look up topics, but not the questions themselves.

Q1: SQL - Build Two Oil Price Tables [15 points] 
---
You are to build a SQL database with two tables, from two oil price files, `brent-daily.csv` and `wti-daily.csv`.

Complete the `feed_data` function which will feed all data from `filename` (argument) into a sqlite database. While doing this, make sure of the following: 
- The SQL tables should not include the first row of the csv files (it has the name of the columns in it.)
- Database name: lab10.db
- Database table names: `brent_oil_price` and `wti_oil_price`
- Columns in each table: date (type: DATE) and price (type: REAL)

For this function, you don't have to print/return anything. Do not change given code.   

Note 1: To read in the csv files, you may use the `csv` or `pandas` libraries.<br>
Note 2: Date column in the csv files will have "YYYY-MM-DD" format.<br>
Note 3: Use the datetime module for date manipulation. References: Lecture on March 26, datetime.ipynb; https://docs.python.org/3/library/datetime.html Specifically: https://docs.python.org/3/library/datetime.html#datetime.datetime.strptime<br>
Note 4: Once you use strptime, you will get datetime. But, you need to insert date. (See `date()` function in datetime)<br>
Note 5: In the print output, if you see a string instead of datetime.date(...), then you do not have the data stored in the correct format.  

In [56]:
from datetime import datetime
import sqlite3
import csv

def feed_data(filename, table_name):
    conn = sqlite3.connect('lab10.db')
    cur = conn.cursor()
    ### Write Your code here
    cur.execute(f"DROP TABLE IF EXISTS {table_name}")
    cur.execute(f'''CREATE TABLE {table_name}( 
    id     INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    Date   DATE,
    Price  REAL)''')

    with open(filename,'r') as file:
        for line in file.readlines()[1:]:
            date,price = line.strip().split(',')
            date_str = datetime.strptime(date,'%Y-%m-%d').date()
            cur.execute(f"INSERT INTO {table_name}(Date,Price) VALUES (?,?)",(date_str,price))
    
    print(filename)
    cur.execute('SELECT * FROM ' + table_name + ' LIMIT 1')
    return_val = cur.fetchall()
    print('  first: ', return_val)

    cur.execute('SELECT * FROM ' + table_name + ' ORDER BY date DESC LIMIT 1')
    return_val = cur.fetchall()
    print('  last:  ', return_val)

    conn.commit()
    conn.close()

In [57]:
# open test
    
feed_data('brent-daily.csv', 'brent_oil_price')
feed_data('wti-daily.csv', 'wti_oil_price')

brent-daily.csv
  first:  [(1, '1987-05-20', 18.63)]
  last:   [(9600, '2025-03-28', 73.63)]
wti-daily.csv
  first:  [(1, '1986-01-02', 25.56)]
  last:   [(9890, '2025-03-28', 69.36)]


Q2: SQL queries [15 points, 3 per query] 
---
For this question, you have to write queries on the `lab10.db` for the following questions and print the results of the query.
1. Get Brent oil prices for March 2025. Print format: [(date, price)]   
2. Number of days for which there exists prices for both (Brent and WTI) oil. Print format: int 
3. Compute the percentage of dates when the Brent oil price is higher than the WTI oil price. Print format: float%
4. Find number of days where Brent oil price is at least 100 and at most 120. Print format: int
5. Get information of Brent and WTI oil price in January-March 2025 which is sorted by difference between Brent and WTI oil price. Print format: [(date, brent oil price, wti oil price)]

In [60]:
# Lab10.Q2.1 Brent March 2025
conn = sqlite3.connect('lab10.db')
cur = conn.cursor()

## Your code here
data = cur.execute("SELECT Date,Price FROM brent_oil_price WHERE strftime('%Y',Date) = '2025' AND strftime('%m',Date) = '03'").fetchall()
print(data)

conn.commit()
conn.close()    

[('2025-03-03', 71.62), ('2025-03-04', 71.04), ('2025-03-05', 69.3), ('2025-03-06', 69.46), ('2025-03-07', 70.36), ('2025-03-10', 69.28), ('2025-03-11', 69.56), ('2025-03-12', 70.95), ('2025-03-13', 69.88), ('2025-03-14', 70.58), ('2025-03-17', 71.07), ('2025-03-18', 70.56), ('2025-03-19', 70.78), ('2025-03-20', 72.0), ('2025-03-21', 72.16), ('2025-03-24', 73.0), ('2025-03-25', 73.02), ('2025-03-26', 73.79), ('2025-03-27', 74.03), ('2025-03-28', 73.63)]


In [61]:
# Lab10.Q2.2 Number of days with parallel Brent-WTI data
conn = sqlite3.connect('lab10.db', detect_types=sqlite3.PARSE_DECLTYPES|sqlite3.PARSE_COLNAMES)
cur = conn.cursor()

## Your code here
same_days = cur.execute("SELECT w.Date FROM wti_oil_price AS w INNER JOIN brent_oil_price AS b ON b.Date = w.Date").fetchall()
print(len(same_days))

# wti_days = cur.execute("SELECT Date FROM wti_oil_price").fetchall()
# brent_days = cur.execute("SELECT Date FROM brent_oil_price").fetchall()
# count_same_days = 0
# for day in wti_days:
#     if day in brent_days:
#         count_same_days += 1

# print(count_same_days)

conn.commit()
conn.close()   

9469


In [81]:
# Lab10.Q2.3 Percentage days with Brent > WTI
conn = sqlite3.connect('lab10.db', detect_types=sqlite3.PARSE_DECLTYPES|sqlite3.PARSE_COLNAMES)
cur = conn.cursor()

## Your code here
same_days = cur.execute("SELECT w.Date,w.Price,b.Price FROM wti_oil_price AS w INNER JOIN brent_oil_price AS b ON b.Date = w.Date").fetchall()

total_days = 0
brent_higher_price_days = 0
for date, wti_price, brent_price in same_days:
    total_days += 1
    if brent_price > wti_price:
        brent_higher_price_days += 1

brent_higher_percent = brent_higher_price_days/ total_days *100
print(round(brent_higher_percent,2))

conn.commit()
conn.close()

42.97


In [63]:
# Lab10.Q2.4 Number of days with Brent in [100 .. 120]
conn = sqlite3.connect('lab10.db', detect_types=sqlite3.PARSE_DECLTYPES|sqlite3.PARSE_COLNAMES)
cur = conn.cursor()

## Your code here
result = cur.execute("SELECT COUNT(Price) FROM brent_oil_price WHERE Price >=100 and Price <=120").fetchone()
print(result[0])

conn.commit()
conn.close()

966


In [65]:
# Lab10.Q2.5 Sorted by Brent - WTI
conn = sqlite3.connect('lab10.db')
cur = conn.cursor()

## Your code here
result = cur.execute("""SELECT w.Date,b.Price,w.Price 
                     FROM wti_oil_price as w
                     LEFT JOIN brent_oil_price AS b ON w.Date = b.Date
                     WHERE w.Date BETWEEN '2025-01' AND '2025-04'
                     ORDER BY (ABS(b.Price - w.Price)) ASC
                     """).fetchall()

print(result)

conn.commit()
conn.close()

[('2025-01-15', 82.03, 80.04), ('2025-01-13', 81.01, 78.82), ('2025-01-14', 79.92, 77.5), ('2025-01-03', 76.51, 73.96), ('2025-01-16', 81.29, 78.68), ('2025-01-06', 76.3, 73.56), ('2025-03-04', 71.04, 68.26), ('2025-01-07', 77.05, 74.25), ('2025-02-03', 75.96, 73.16), ('2025-01-02', 75.93, 73.13), ('2025-01-08', 76.16, 73.32), ('2025-01-17', 80.79, 77.88), ('2025-03-05', 69.3, 66.31), ('2025-01-09', 76.92, 73.92), ('2025-03-06', 69.46, 66.36), ('2025-01-10', 79.76, 76.57), ('2025-03-03', 71.62, 68.37), ('2025-03-10', 69.28, 66.03), ('2025-03-12', 70.95, 67.68), ('2025-03-11', 69.56, 66.25), ('2025-03-07', 70.36, 67.04), ('2025-03-13', 69.88, 66.55), ('2025-03-14', 70.58, 67.18), ('2025-01-21', 79.29, 75.89), ('2025-02-28', 73.18, 69.76), ('2025-03-17', 71.07, 67.58), ('2025-02-04', 76.2, 72.7), ('2025-02-10', 75.87, 72.32), ('2025-01-22', 79.0, 75.44), ('2025-02-05', 74.61, 71.03), ('2025-03-19', 70.78, 67.16), ('2025-02-07', 74.66, 71.0), ('2025-03-18', 70.56, 66.9), ('2025-01-23', 78

### A bonus question might be added after the lecture on Friday, April 11 depending on progress on Pandas.